In [846]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('whitegrid')

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In this notebook, we explore possible improvements upon the models introduced earlier, as well as assess the effectiveness of new models. In both cases, much anticipation is built upon the inclusion of the 2021 census data.

In [847]:
#load the restaurant dataset with census features
rd_train = pd.read_csv("/Users/dominiquekemp/Documents/GitHub/'Will It Restaurant?'/final_level1_train_data.csv")

#load the list of features deemed most impactful
with open("/Users/dominiquekemp/Documents/GitHub/'Will It Restaurant?'/final_features.csv", "r") as file:
    next(file)
    features = [line.strip() for line in file]
    
features

['latitude',
 'longitude',
 'photos_count',
 'reviews',
 'Median Household Income',
 'Education Level %',
 '% Black',
 '% Hispanic/Latino',
 'Foreign Born Immigrant %',
 'Median Age']

We remind the reader that this dataset only presents for the restaurants of price level 1.

In [848]:
#check for NA values (in order to run the regressions)
rd_train.isna().sum(axis = 0)

name                          0
site                          0
subtypes                      0
category                      0
type                          0
phone                         0
full_address                  0
borough                       0
street                        1
postal_code                   0
latitude                      0
longitude                     0
h3                            0
rating                        0
reviews                       0
reviews_link                  0
reviews_per_score_1           0
reviews_per_score_2           0
reviews_per_score_3           0
reviews_per_score_4           0
reviews_per_score_5           0
photos_count                  0
photo                         0
working_hours_old_format     26
other_hours                 703
business_status               0
about                         0
range                         0
description                 450
booking_appointment_link      0
location_link                 0
location

In [849]:
#drop rows with missing "Median Household Income" values
rd_train.dropna(axis = 0, subset = 'Median Household Income', inplace = True, ignore_index= True)

In this notebook, we shall explore three new models: 
1) 'K Neighbors Regression' with all of the features listed in "features_importances.csv" that have importance higher than 0.05;

2) 'Random Forest Regression' with those same features;

3) 'Random Forest Regression with K Neighbors Boost'


and we shall also develop a new form of clustering aimed at improved estimates in subsets of the data misfitted by all models including the above. This clustering will occur manually, by way of decision tree classification.

## Model 1 (K Neighbors Regression)

In [850]:
#make stratified k-fold object
n_splits = 5
kfold = StratifiedKFold(n_splits, shuffle = True, random_state= 5501)

In [851]:
#prepare the index set for 'rmse' dataframe tracking value of k and borough of each restaurant
boroughs = np.sort(rd_train.borough.unique())
num_k = 20

many_boroughs = np.array([])
many_ks = []
for i in range(num_k): 
    many_boroughs = np.concatenate((boroughs, many_boroughs))
    many_ks = many_ks + len(boroughs)*[i]


In [852]:
features1 = features[:7]


In [853]:
features1

['latitude',
 'longitude',
 'photos_count',
 'reviews',
 'Median Household Income',
 'Education Level %',
 '% Black']

In [854]:

rmse = np.zeros((num_k, n_splits))

rmse_df = pd.DataFrame({"borough": many_boroughs, "k": many_ks})
for k in range(num_k):
    knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsRegressor(k+1))])
    i = 0
    for train_index, test_index in kfold.split(rd_train, rd_train.borough):
         rd_t_train= rd_train.iloc[train_index]
         rd_t_test = rd_train.iloc[test_index]

         knn_pipe.fit(rd_t_train[features1], rd_t_train.rating)
         preds = knn_pipe.predict(rd_t_test[features1])
         rmse[k, i] = root_mean_squared_error(rd_t_test.rating, preds)
         
         for bor in rd_t_test.borough.unique():
              rd_t_bor = rd_t_test[rd_t_test.borough == bor]
              bor_preds = knn_pipe.predict(rd_t_bor[features1])
              rmse_df.loc[(rmse_df.borough == bor) &(rmse_df.k == k), i] = root_mean_squared_error(rd_t_test[rd_t_test.borough == bor].rating,
                                                                               bor_preds)
         
         i += 1

/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least

In [855]:


#add column with average of the available rmse's
rmse_df['Avg. RMSE'] = rmse_df.iloc[:, -5:].sum(axis = 1)/(n_splits - rmse_df.isna().sum(axis = 1))

#find the best k for each borough
def find_k(data, borough):
    df = data[data.borough == borough]
    return np.argmin(df['Avg. RMSE']) + 1

best_ks = [find_k(rmse_df, bor) for bor in boroughs]

min_rmses = [rmse_df.loc[(rmse_df.k == best_ks[i] -1) & 
                         (rmse_df.borough == boroughs[i]), 'Avg. RMSE'].reset_index(drop=True)[0] for i in range(len(boroughs))]


#give the desired dataframe of rmse with best k-neighbors model for each borough
rmse = np.mean(rmse, axis = 1)
min_k = np.argmin(rmse)
gen_k_neigh = pd.DataFrame({'Borough': boroughs, "Best k": best_ks, "CV Avg. RMSE": min_rmses})
print(f"The overall CV avg. RMSE is {rmse[min_k]: .3f} with k = {min_k + 1}. The dataframe below presents the best k-neighbors results individualized to each borough.")
gen_k_neigh['Number of Restaurants'] = [len(rd_train[rd_train.borough == bor]) for bor in boroughs]
gen_k_neigh

The overall CV avg. RMSE is  0.440 with k = 18. The dataframe below presents the best k-neighbors results individualized to each borough.


,Borough,Best k,CV Avg. RMSE,Number of Restaurants
0,Allegheny West,16,0.350360,9
1,Bella Vista,14,0.286226,9
2,Bensalem,19,0.538596,3
3,Center City,12,0.430627,25
4,Center City East,20,0.442763,49
5,Chestnut Hill,10,0.304538,8
6,East Germantown,4,0.382769,13
7,East Kensington,16,0.085417,3
8,East Passyunk Crossing,5,0.282390,6
9,Elmwood Park,19,0.324644,18


## Model 2 (Random Forest Regression)

In [856]:
#run random forest model on the dataset with same features                                  

random_for = RandomForestRegressor(n_estimators=100, max_depth = 4, random_state=59038, max_features = len(features))
rmse_bor = pd.DataFrame(index = boroughs)
rmse = np.zeros(n_splits)
i = 0
for train_index, test_index in kfold.split(rd_train, rd_train.borough):
    rd_t_train= rd_train.iloc[train_index]
    rd_t_test = rd_train.iloc[test_index]

    random_for.fit(rd_t_train[features1], rd_t_train.rating)
    preds = random_for.predict(rd_t_test[features1])
    rmse[i] = root_mean_squared_error(rd_t_test.rating, preds)
         
    for bor in rd_t_test.borough.unique():
        rd_t_bor = rd_t_test[rd_t_test.borough == bor]
        bor_preds = random_for.predict(rd_t_bor[features1])
        rmse_bor.loc[bor, i] = root_mean_squared_error(rd_t_test[rd_t_test.borough == bor].rating,
                                                                               bor_preds)
         
    i += 1



print(f"The overall CV avg. RMSE is {np.mean(rmse): .3f}.")
rmse_bor.mean(axis = 1)

/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


The overall CV avg. RMSE is  0.436.


Allegheny West             0.372813
Bella Vista                0.290395
Bensalem                   0.627050
Center City                0.453729
Center City East           0.439015
Chestnut Hill              0.347609
East Germantown            0.418608
East Kensington            0.130975
East Passyunk Crossing     0.381582
Elmwood Park               0.303195
Essington                  0.540986
Fairhill                   0.392209
Fairmount                  0.310260
Feltonville                0.303520
Fishtown                   0.309104
Fox Chase                  0.411625
Frankford                  0.280205
Germantown                 0.308672
Harrowgate                 0.222498
Kingsessing                0.292379
Lester                     0.394069
Lower Moyamensing          0.230159
Manayunk                   0.348218
Mount Airy                 0.438567
Newbold                    0.403814
Nicetown–Tioga             0.501395
North Philadelphia         0.444015
North Philadelphia East    0

## Model 3 (Random Forest with K Neighbors Boost)

In [857]:
features2 = features1.copy()
features2.remove('% Black')

num_k = 30
rmse = np.zeros((num_k, n_splits))

rmse_df = pd.DataFrame({"borough": many_boroughs, "k": many_ks})
for k in range(num_k):
    knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsRegressor(k+1))])
    i = 0
    for train_index, test_index in kfold.split(rd_train, rd_train.borough):
        rd_t_train= rd_train.iloc[train_index]
        rd_t_test = rd_train.iloc[test_index]
        random_for.fit(rd_t_train[features1], rd_t_train.rating)
        preds_tr = random_for.predict(rd_t_train[features1])
        resids_tr = rd_t_train.rating - preds_tr
        knn_pipe.fit(rd_t_train[features2], resids_tr)

        preds = random_for.predict(rd_t_test[features1])
        new_resids = knn_pipe.predict(rd_t_test[features2])
        rmse[k, i] = root_mean_squared_error(rd_t_test.rating, new_resids + preds)
         
        for bor in rd_t_test.borough.unique():
            rd_t_bor = rd_t_test[rd_t_test.borough == bor]
            bor_preds = random_for.predict(rd_t_bor[features1])
            bor_resids = knn_pipe.predict(rd_t_bor[features2])
            rmse_df.loc[(rmse_df.borough == bor) &(rmse_df.k == k), i] = root_mean_squared_error(rd_t_bor.rating,
                                                                               bor_preds + bor_resids)
         
        i += 1

/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least

In [858]:


#add column with average of the available rmse's
rmse_df['Avg. RMSE'] = rmse_df.iloc[:, -5:].sum(axis = 1)/(n_splits - rmse_df.isna().sum(axis = 1))

#find the best k
best_ks = [find_k(rmse_df, bor) for bor in boroughs]

min_rmses = [rmse_df.loc[(rmse_df.k == best_ks[i] -1) & 
                         (rmse_df.borough == boroughs[i]), 'Avg. RMSE'].reset_index(drop=True)[0] for i in range(len(boroughs))]


#give the desired dataframe
rmse = np.mean(rmse, axis = 1)
min_k = np.argmin(rmse)
gen_k_neigh_boost = pd.DataFrame({'Borough': boroughs, "Best k": best_ks, "CV Avg. RMSE": min_rmses})
print(f"The overall CV avg. RMSE is {rmse[min_k]: .3f} with k = {min_k + 1}.")
gen_k_neigh_boost['Number of Restaurants'] = [len(rd_train[rd_train.borough == bor]) for bor in boroughs]
gen_k_neigh_boost

The overall CV avg. RMSE is  0.440 with k = 29.


,Borough,Best k,CV Avg. RMSE,Number of Restaurants
0,Allegheny West,20,0.329755,9
1,Bella Vista,2,0.284016,9
2,Bensalem,8,0.643545,3
3,Center City,1,0.420236,25
4,Center City East,19,0.452137,49
5,Chestnut Hill,10,0.340694,8
6,East Germantown,20,0.432569,13
7,East Kensington,2,0.110087,3
8,East Passyunk Crossing,12,0.337783,6
9,Elmwood Park,19,0.291469,18


## Model Comparisons

Let us compare this with all of our 'k neighbors' models.

Recall for below that the first 'k neighbors' model applies to all price level 1 restaurants throughout Philly with features of latitude/longitude, reviews, photos count, and cuisine specification. The second model similarly applies to all level 1 restaurants with only features of latitude/longitude taken, while the third relies on the same features yet only picks 'k'-many neighbors within the same borough as the data point tested.

In [859]:

old_rmse = pd.read_csv("/Users/dominiquekemp/Documents/GitHub/'Will It Restaurant?'/level1_modelcomparison.csv")
old_rmse.index = boroughs

old_rmse

,Best k,CV Avg. RMSE,Best k.1,CV Avg. RMSE.1,Best k.2,Min. RMSE,Number of Restaurants
Allegheny West,38,0.269954,5,0.317933,6,0.385603,10
Bella Vista,1,0.242120,4,0.192432,5,0.230552,9
Bensalem,2,0.166667,48,0.519444,0,1.000000,3
Center City,12,0.375324,21,0.457391,5,0.419394,25
Center City East,2,0.414297,50,0.454809,11,0.431824,49
Chestnut Hill,1,0.369610,38,0.372912,6,0.412998,8
East Germantown,15,0.302769,22,0.420062,10,0.422507,13
East Kensington,2,0.083333,16,0.118750,0,1.000000,3
East Passyunk Crossing,1,0.284721,13,0.351479,4,0.413310,6
Elmwood Park,10,0.344593,4,0.334522,11,0.359574,18


In [860]:
#write all results to file for visual analysis
df = pd.concat([old_rmse.iloc[:, :-1], gen_k_neigh.set_index('Borough').iloc[:, :2], 
                rmse_bor.mean(axis = 1).rename('RF CV RMSE'), gen_k_neigh_boost.set_index('Borough').iloc[:, :3]], axis = 1)

df.to_csv("/Users/dominiquekemp/Documents/GitHub/'Will It Restaurant?'/allmodels_comparison(level1).csv")

If we only look at overall CV avg. RMSE, all of these models seem to perform similarly. However, k has to be taken rather large for k neighbors regression, so random forest regression seems most efficient. 

But we note that a subset of the models pursued produce better estimates for certain boroughs where other models perform badly. 

Thus, it seems further confirmed that we could improve our estimates if we cluster.

## Clustering

First, we present evidence showing the benefit for clustering. Choosing for each borough the model that performs best there, we obtain a significantly lower avg. CV RMSE than attained at any previous point.

In [861]:
#combine all models for '$' restaurants

rf_rmse = rmse_bor.mean(axis = 1)
sum = 0
for bor in boroughs:
    array1 = old_rmse.loc[bor, ['CV Avg. RMSE', 'CV Avg. RMSE.1', 'Min. RMSE']].values
    array2 = [rf_rmse[bor]]
    tally = old_rmse.loc[bor, 'Number of Restaurants']
    err = np.min(np.concatenate((array1, array2)))
    sum += tally*err**2
total = old_rmse['Number of Restaurants'].sum()
np.sqrt(sum/total)


np.float64(0.35397367569012683)

For the future, we shall certainly organize our data into clusters by boroughs. However, this is only one tier in our overall clustering strategy. The next tier, introduced in the next section, is a form of clustering that occurs within boroughs- those boroughs whose dining culture is so complex as to encompass widely differing trends.

### Clustering for West Philly

'West Philadelphia' is one of the boroughs where our analysis has persistently performed the worst. This fact seems to imply that there is greater complexity to West Philly than what one single regression model can grasp. Therefore, we use its corresponding dataset to demonstrate the new clustering scheme.

In [862]:
features_wp = ['reviews', 'longitude', 'latitude', 'photos_count', '% Black', 'Education Level %', 'Foreign Born Immigrant %']

The clustering scheme we have in mind is essentially a 'greedy algorithm'. First, we check for the best performing model. And then we assess where the model performs badly, obtaining a feature-based description of this set. Finally, we test various models on this set and select the one that performs best there.

In [864]:
west_ph = rd_train[(rd_train.borough == 'West Philadelphia')]
west_ph = pd.concat([west_ph, pd.get_dummies(west_ph.type)], axis = 1)
features_wp = features_wp + west_ph.columns[52:].to_list()

In [865]:

rmse_wp = np.zeros(n_splits)
rf_wp = RandomForestRegressor(max_depth = 4, random_state=59038, max_features = len(features))
kfold1 = KFold(n_splits = 5, random_state= 5501, shuffle=True)
i=0
for train_index, test_index in kfold1.split(west_ph):
    train_set = west_ph.iloc[train_index]
    test_set = west_ph.iloc[test_index]
    rf_wp.fit(train_set[features_wp], train_set.rating)

    preds = rf_wp.predict(test_set[features_wp])
    rmse_wp[i] = root_mean_squared_error(test_set.rating, preds)
    i+=1

np.mean(rmse_wp)

np.float64(0.5078397461481556)

In [866]:
score_df = pd.DataFrame({'feature': features_wp,
                            'importance_score': rf_wp.feature_importances_})
score_df.sort_values('importance_score', ascending = False, inplace = True)
score_df.head(10)

,feature,importance_score
2,latitude,0.134554
1,longitude,0.120944
0,reviews,0.115511
5,Education Level %,0.112408
4,% Black,0.110100
3,photos_count,0.091860
20,Fast food restaurant,0.084980
6,Foreign Born Immigrant %,0.071592
29,Pizza delivery,0.057203
27,Mexican restaurant,0.025578


In [599]:
features_wp

['reviews',
 'longitude',
 'latitude',
 'photos_count',
 'Afghan restaurant',
 'African restaurant',
 'American restaurant',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Association / Organization',
 'Bagel shop',
 'Bakery',
 'Bar',
 'Bar & grill',
 'Brazilian restaurant',
 'Breakfast restaurant',
 'Brunch restaurant',
 'Bubble tea store',
 'Cafe',
 'Cafeteria',
 'Cambodian restaurant',
 'Caribbean restaurant',
 'Cheesesteak restaurant',
 'Chicken restaurant',
 'Chicken wings restaurant',
 'Chinese restaurant',
 'Chinese takeaway',
 'Cocktail bar',
 'Coffee shop',
 'Colombian restaurant',
 'Convenience store',
 'Creperie',
 'Deli',
 'Dessert shop',
 'Dim sum restaurant',
 'Diner',
 'Dominican restaurant',
 'Donut shop',
 'Eritrean restaurant',
 'Ethiopian restaurant',
 'Falafel restaurant',
 'Fast food restaurant',
 'Fish & chips restaurant',
 'French restaurant',
 'Fried chicken takeaway',
 'Gastropub',
 'Greek restaurant',
 'Grill',
 'Halal restaurant',
 'Health food restaurant

In [867]:
data = pd.concat([rd_train, pd.get_dummies(rd_train['type'])], axis = 1)

features_wp = features_wp[:4] + data.columns[52:].tolist()

The best model that we have for "West Philadelphia" yet remains (general)'k-neighbors' regression with geospatial, 'reviews', and 'photos_count' features plus cuisine types. The CV avg. RMSE in ratings here is about 0.45. Let us see whether we can improve on this by the manual clustering we've promised.

In [868]:
#a function for obtaining the errors of the predictions made by an inputted model
def get_errs(model, features, data, bor, kfold):

    errs = np.array([])
    indices = np.array([])
    

    
    if bor is not None:
        for train_index, test_index in kfold.split(data, data.borough):
            train_set = data.iloc[train_index]
            test_set = data.iloc[test_index]
            model.fit(train_set[features], train_set.rating)

            test_set_bor = test_set[test_set.borough == bor]
            bor_test = test_set_bor.index.to_numpy()
            preds = model.predict(test_set_bor[features])
    
            diff = test_set_bor.rating - preds
            errs = np.concatenate((errs, np.abs(diff)))
            indices = np.concatenate((indices, bor_test))
        return errs, indices
    else:
        for train_index, test_index in kfold.split(data):
            train_set = data.iloc[train_index]
            test_set = data.iloc[test_index]
            model.fit(train_set[features], train_set.rating)
            preds = model.predict(test_set[features])
            index = test_set.index.to_numpy()

            diff = test_set.rating - preds
            errs = np.concatenate((errs, np.abs(diff)))
            indices = np.concatenate((indices, index))
        return errs, indices





Now we have the first step of the clustering algorithm.

In [869]:
#apply 'get_errs' with the best model
knn_wp =Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsRegressor(8))])
        
   
errs, indices = get_errs(knn_wp, features_wp, data, 'West Philadelphia', kfold)

/opt/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [870]:
np.sum((errs > 0.5))

np.int64(30)

In [871]:
#a function that assigns binary class labels to unacceptable errors and acceptable errors
def assign_class(data, bor, indices, errs, thresh):
    if bor is not None:
        data = data[data.borough == bor]
    
    data.loc[indices, 'Errors'] = errs #assign error values to the dataframe
    #classify large errors
    data['Class'] = data['Errors'].map(lambda x: x > thresh).astype(int)
    return data


#apply 'assign_class' to west philly
wp = assign_class(data, 'West Philadelphia', indices, errs, thresh = 0.5)


/var/folders/3j/bbm3n5gj58xg3wkn3ngyvyrm0000gn/T/ipykernel_76077/1468260896.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[indices, 'Errors'] = errs #assign error values to the dataframe
/var/folders/3j/bbm3n5gj58xg3wkn3ngyvyrm0000gn/T/ipykernel_76077/1468260896.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Class'] = data['Errors'].map(lambda x: x > thresh).astype(int)


In [609]:
features

['latitude',
 'longitude',
 'photos_count',
 'reviews',
 'Median Household Income',
 'Education Level %',
 '% Black',
 '% Hispanic/Latino',
 'Foreign Born Immigrant %',
 'Median Age']

Now that we have classified the large errors, we attempt decision tree classification to locate them by feature description. For the latter, we will use all available features.

In [872]:
#gather all features for the classification
class_features = features + ['Total Population', 'Poverty Rate %', '% White', '% Asian', 'Neighborhood Turnover %'] + features_wp[4:]

In [873]:
#import modules necessary for decision tree classification 
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

from tree_helpers import SkNode, create_sk_nodes #tree_helpers is a Python file that organizes the nodes of a decision tree and returns the feature constraints of the leaf nodes
from sklearn.metrics import precision_score, recall_score



In [874]:
wp.loc[wp.Class == 1]

,name,site,subtypes,category,type,phone,full_address,borough,street,postal_code,...,Takeout Restaurant,Tex-Mex restaurant,Thai restaurant,Traditional American restaurant,Vegan restaurant,Vegetarian restaurant,Vietnamese restaurant,West African restaurant,Errors,Class
28,CM Chicken,True,"Chicken restaurant, Restaurant",restaurants,Chicken restaurant,True,"3180 Chestnut St, Philadelphia, PA 19104",West Philadelphia,3180 Chestnut St,19104.0,...,False,False,False,False,False,False,False,False,0.8500,1
75,Kabobeesh Pizza & Grill,True,Pizza restaurant,restaurants,Pizza restaurant,True,"5206 Chestnut St, Philadelphia, PA 19139",West Philadelphia,5206 Chestnut St,19139.0,...,False,False,False,False,False,False,False,False,0.6000,1
91,Papa Johns Pizza,True,Pizza restaurant,restaurants,Pizza restaurant,True,"104 S 40th St, Philadelphia, PA 19104",West Philadelphia,104 S 40th St,19104.0,...,False,False,False,False,False,False,False,False,0.9250,1
104,Fresh Donuts,True,"Donut shop, Breakfast restaurant",Donut shop,Donut shop,True,"3914 Lancaster Ave, Philadelphia, PA 19104",West Philadelphia,3914 Lancaster Ave,19104.0,...,False,False,False,False,False,False,False,False,0.5875,1
147,El Taco,True,"Mexican restaurant, Burrito restaurant, Taco r...",restaurants,Mexican restaurant,True,"3233 Powelton Ave, Philadelphia, PA 19104",West Philadelphia,3233 Powelton Ave,19104.0,...,False,False,False,False,False,False,False,False,0.5375,1
183,Cucina Zapata,False,Mexican restaurant,restaurants,Mexican restaurant,False,"3101 Ludlow St, Philadelphia, PA 19104",West Philadelphia,3101 Ludlow St,19104.0,...,False,False,False,False,False,False,False,False,0.9625,1
197,Raising Cane's Chicken Fingers,True,"Fast food restaurant, American restaurant, Chi...",restaurants,Fast food restaurant,True,"3925 Walnut St, Philadelphia, PA 19104",West Philadelphia,3925 Walnut St,19104.0,...,False,False,False,False,False,False,False,False,0.5750,1
219,Chipotle Mexican Grill,True,"Mexican restaurant, Caterer, Fast food restaurant",restaurants,Mexican restaurant,True,"3925 Walnut St, Philadelphia, PA 19104",West Philadelphia,3925 Walnut St,19104.0,...,False,False,False,False,False,False,False,False,0.8125,1
242,ASAD’S HOT CHICKEN,True,Fast food restaurant,restaurants,Fast food restaurant,True,"4627 Woodland Ave, Philadelphia, PA 19143",West Philadelphia,4627 Woodland Ave,19143.0,...,False,False,False,False,False,False,False,False,1.4625,1
299,Dahlak,True,"Eritrean restaurant, Bar, Ethiopian restaurant...",restaurants,Eritrean restaurant,True,"4708 Baltimore Ave, Philadelphia, PA 19143",West Philadelphia,4708 Baltimore Ave,19143.0,...,False,False,False,False,False,False,False,False,0.6125,1


In [878]:
#we write the classification algorithm as a class
class LE_Class():
    def __init__(self, class_features):
        self.features = class_features
        l = len(class_features)
        self.feat_imp = pd.DataFrame({'features': class_features, 'importance score': l*[0]})
        self.tree = None
        self.prec = 0
        self.recall = 0

    def fit(self, X):
        self.tree = DecisionTreeClassifier(
            min_samples_leaf = 5, # minimum number of samples in each leaf, to prevent overfitting
            random_state= 216)

        self.tree.fit(X[self.features], X['Class'])
        preds = self.tree.predict(X[self.features])
        self.prec = np.round(precision_score(X['Class'], preds), 4)
        self.recall = np.round(recall_score(X['Class'], preds), 4)

        self.feat_imp = pd.DataFrame({'features': class_features, 'importance score': self.tree.feature_importances_}).sort_values(
            'importance score', ascending = False)


In [879]:
#apply the classification to west philly data in class-labeled form
wp_le = LE_Class(class_features)
wp_le.fit(wp)
nodes = create_sk_nodes(wp_le.tree)

print(wp_le.prec)
print(wp_le.recall)
print(wp_le.feat_imp.head(10))


0.75
0.9
                            features  importance score
1                          longitude          0.338381
3                            reviews          0.180635
52              Fast food restaurant          0.131541
2                       photos_count          0.110872
8           Foreign Born Immigrant %          0.089437
0                           latitude          0.082086
82                  Pizza restaurant          0.067048
101  Traditional American restaurant          0.000000
102                 Vegan restaurant          0.000000
76         Middle Eastern restaurant          0.000000


As the next step, we extract the classification rubric provided by LE_Class, so that we can locate the corresponding cluster in the test dataset.

In [880]:
#a function for obtaining the Cartesian description for the data subsets corresponding to the leaf nodes
def get_subsets(nodes):
    #collect the coordinate constraints implicated for each leaf node
    endpts = {}
    i=0
    for node in nodes.values():
        if (node.is_leaf) and (node.prediction == 1):
            endpts[i] = node.get_constraints()
            i += 1
    num_subsets = len(endpts)

    #rearrange to the corresponding Cartesian product of coordinate sets for each leaf
    subsets = {key: [] for key in range(num_subsets)}
    i = 0
    for coord_bds in endpts.values():
        for j in range(len(coord_bds[0])):
            list = [coord_bds[0][j], coord_bds[1][j]]
            subsets[i].append(list)
        i += 1
    return subsets

subsets = get_subsets(nodes)
num_subsets = len(subsets)

#locate the cluster within the dataframe
def def_cluster(data, subsets, class_features):
    cluster = {}
    feature_num = len(class_features)


    for i in range(num_subsets):
        cluster[i] = data
        for j in range(feature_num):
            feature = class_features[j]
            cluster[i] = cluster[i].loc[(cluster[i][feature] > subsets[i][j][0]) &
                                 (cluster[i][feature] < subsets[i][j][1])]

    whole = pd.DataFrame({})
    for i in range(num_subsets):
        whole = pd.concat([whole, cluster[i]])
    return whole

whole = def_cluster(wp, subsets, class_features)
whole

,name,site,subtypes,category,type,phone,full_address,borough,street,postal_code,...,Takeout Restaurant,Tex-Mex restaurant,Thai restaurant,Traditional American restaurant,Vegan restaurant,Vegetarian restaurant,Vietnamese restaurant,West African restaurant,Errors,Class
329,Pete's Pizza,True,"Pizza restaurant, Italian restaurant, Takeout ...",restaurants,Pizza restaurant,True,"1913 N 54th St, Philadelphia, PA 19131",West Philadelphia,1913 N 54th St,19131.0,...,False,False,False,False,False,False,False,False,1.0375,1
809,Larry's Steaks,True,"Pizza restaurant, Cheesesteak restaurant, Fast...",restaurants,Pizza restaurant,True,"2459 N 54th St, Philadelphia, PA 19131",West Philadelphia,2459 N 54th St,19131.0,...,False,False,False,False,False,False,False,False,0.1250,0
812,Little Caesars Pizza,True,"Pizza restaurant, Fast food restaurant, Pizza ...",restaurants,Pizza restaurant,True,"5901 Lancaster Ave, Philadelphia, PA 19151",West Philadelphia,5901 Lancaster Ave,19151.0,...,False,False,False,False,False,False,False,False,0.5000,0
850,Shalom Pizzeria,True,"Pizza restaurant, Dairy, Falafel restaurant, F...",restaurants,Pizza restaurant,True,"7598 Haverford Ave, Philadelphia, PA 19151",West Philadelphia,7598 Haverford Ave,19151.0,...,False,False,False,False,False,False,False,False,0.6125,1
944,Pete's Pizza,True,"Pizza restaurant, Delivery Restaurant",restaurants,Pizza restaurant,True,"5158 Haverford Ave, Philadelphia, PA 19139",West Philadelphia,5158 Haverford Ave,19139.0,...,False,False,False,False,False,False,False,False,0.9000,1
46,Class of 1920 Commons,True,"Restaurant, Coffee shop, Cafeteria",Cafeteria,Restaurant,True,"3800 Locust Walk, Philadelphia, PA 19104",West Philadelphia,3800 Locust Walk,19104.0,...,False,False,False,False,False,False,False,False,0.1500,0
91,Papa Johns Pizza,True,Pizza restaurant,restaurants,Pizza restaurant,True,"104 S 40th St, Philadelphia, PA 19104",West Philadelphia,104 S 40th St,19104.0,...,False,False,False,False,False,False,False,False,0.9250,1
219,Chipotle Mexican Grill,True,"Mexican restaurant, Caterer, Fast food restaurant",restaurants,Mexican restaurant,True,"3925 Walnut St, Philadelphia, PA 19104",West Philadelphia,3925 Walnut St,19104.0,...,False,False,False,False,False,False,False,False,0.8125,1
324,Amsale Cafe,True,Ethiopian restaurant,restaurants,Ethiopian restaurant,True,"4817 Walnut St, Philadelphia, PA 19139",West Philadelphia,4817 Walnut St,19139.0,...,False,False,False,False,False,False,False,False,0.6250,1
337,Royal Pizza,True,Pizza restaurant,restaurants,Pizza restaurant,True,"4200 Baltimore Ave, Philadelphia, PA 19104",West Philadelphia,4200 Baltimore Ave,19104.0,...,False,False,False,False,False,False,False,False,0.6750,1


We write functions that determine the rmse's on the 'good' and 'bad' subsets of the West Philly data.

In [881]:
def good_rmse(data, amb_data):

    index_to_remove = set(data.index.tolist())
    index = set(amb_data.index.tolist())
    index = index.difference(index_to_remove)
    index = [*index]


    squares = amb_data.loc[index, 'Errors']**2
    return np.sqrt(squares.mean())

good_rmse(whole, wp)

np.float64(0.27477854128961154)

In [882]:
def bad_rmse(data):

    squares = data['Errors']**2
    return np.sqrt(squares.mean())

bad_rmse(whole)

np.float64(0.7054660102844549)

It will be our objective to reduce the rmse on "whole" as much as possible.

### Modeling on the New West Philly Cluster

In [883]:
wp_le.feat_imp.head(10)

,features,importance score
1,longitude,0.338381
3,reviews,0.180635
52,Fast food restaurant,0.131541
2,photos_count,0.110872
8,Foreign Born Immigrant %,0.089437
0,latitude,0.082086
82,Pizza restaurant,0.067048
101,Traditional American restaurant,0.000000
102,Vegan restaurant,0.000000
76,Middle Eastern restaurant,0.000000


In [884]:
#pick the most significant features returned by LE_Class
w_feats = ['latitude', 'longitude', 'photos_count', 'Foreign Born Immigrant %', 'reviews', 'Pizza restaurant', 'Fast food restaurant']

In [889]:
num_k = 20

w_rmse = np.zeros((num_k, n_splits))

for k in range(num_k):
    knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsRegressor(k+1))])
    i=0
    for train, test in kfold1.split(whole):
        train_set = whole.iloc[train]
        test_set = whole.iloc[test]

        knn_pipe.fit(train_set[w_feats[:-5] ], train_set.rating)
        preds = knn_pipe.predict(test_set[w_feats[:-5] ])
        w_rmse[k, i] = root_mean_squared_error(test_set.rating, preds)
        i += 1
w_rmse = np.mean(w_rmse, axis = 1)
min_k = np.argmin(w_rmse)
print(f"The best CV avg. RMSE is {w_rmse[min_k]: .3f}, occurring at k = {min_k +1}.")


The best CV avg. RMSE is  0.648, occurring at k = 6.


We try some other models with various combinations of features.

In [648]:
whole.type.value_counts()

type
Pizza restaurant        10
Fast food restaurant     5
Restaurant               4
Mexican restaurant       4
Ethiopian restaurant     2
Halal restaurant         2
Chicken restaurant       2
Falafel restaurant       1
Sandwich shop            1
Bagel shop               1
Chinese restaurant       1
Coffee shop              1
Eritrean restaurant      1
Indian restaurant        1
Donut shop               1
Name: count, dtype: int64

In [759]:
features

['latitude',
 'longitude',
 'photos_count',
 'reviews',
 'Median Household Income',
 'Education Level %',
 '% Black',
 '% Hispanic/Latino',
 'Foreign Born Immigrant %',
 'Median Age']

In [890]:
mlr = LinearRegression()
mlr_rmse = np.zeros(n_splits)
i=0 

for train, test in kfold1.split(whole):
    whole_tr = whole.iloc[train]
    whole_te = whole.iloc[test]
    mlr.fit(whole_tr[w_feats[:-5] + ['Poverty Rate %']], whole_tr.rating)
    preds = mlr.predict(whole_te[w_feats[:-5] + ['Poverty Rate %']])

    mlr_rmse[i] = root_mean_squared_error(whole_te.rating, preds)
    i += 1
np.mean(mlr_rmse)


np.float64(0.6260908604272559)

None of our models are providing much reduction in RMSE. We try a different perspective now.

In [897]:
rf = RandomForestRegressor(n_estimators=100, max_depth = 7, random_state=59038)
w_rmse = np.zeros(n_splits)
i=0
for train, test in kfold1.split(whole):
    train_set = whole.iloc[train]
    test_set = whole.iloc[test]

    rf.fit(train_set[class_features], train_set.rating)
    preds = rf.predict(test_set[class_features])
    w_rmse[i] = root_mean_squared_error(test_set.rating, preds)
    i += 1

np.mean(w_rmse)


np.float64(0.6213705452827143)

In [898]:
rf.feature_importances_

pd.DataFrame({"features": class_features, 'importance scores': rf.feature_importances_}).sort_values("importance scores", ascending = False).head(10)

,features,importance scores
2,photos_count,0.228796
10,Total Population,0.211553
1,longitude,0.103237
0,latitude,0.097645
3,reviews,0.060217
9,Median Age,0.052146
11,Poverty Rate %,0.041185
7,% Hispanic/Latino,0.039690
4,Median Household Income,0.031021
5,Education Level %,0.029662


In [901]:
new_feats = ['latitude', 'longitude', 'reviews', 'photos_count', 'Median Age', 'Total Population', 'Poverty Rate %']

The point here was not to evaluate the predictive ability of random forest regression here, but rather to use it as a means of feature selection. We try those features with linear regression now.

In [904]:
mlr = LinearRegression()
mlr_rmse = np.zeros(n_splits)
i=0 

for train, test in kfold1.split(west_ph):
    west_ph_tr = west_ph.iloc[train]
    west_ph_te = west_ph.iloc[test]
    mlr.fit(west_ph_tr[new_feats], west_ph_tr.rating)
    preds = mlr.predict(west_ph_te[new_feats])
    for j in range(len(preds)):
        value = preds[j]
        if value >= 5.0:
            preds[j] = 5.0

    mlr_rmse[i] = root_mean_squared_error(west_ph_te.rating, preds)
    i += 1
np.mean(mlr_rmse)


np.float64(0.49509203828066434)

This is our best avg. CV RMSE yet, sizably better than the original value of 0.71.

We write the defining characteristics of the new cluster to file.

In [905]:
sub_index = [key for key in subsets.keys()]

cluster_df = pd.DataFrame(columns = ['origin', 'good_model', 'bad_model', 'class_features']+ sub_index)

cluster_df.loc[0] = [('West Philadelphia', '1'), ('8-neighbors (gen.)', features_wp), ('Linear Regression', new_feats), class_features] + [subset for subset in subsets.values()]

In [906]:
cluster_df.to_csv("../'Will It Restaurant?'/clusters.csv", index = False)